In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Expected labels (columns)
EXPECTED_LABELS = [
    "کد آگهی", "منطقه", "محله", "متراژ", "قیمت کل", "طبقه", "تعداد اتاق", 
    "کل طبقات", "تعداد واحد در طبقه", "موقعیت", "نوع ساخت",
    "سال ساخت", "نوع سند", "زمان تحویل", "متراژ گذر", "عرض قطعه",
    "متراژ زمین"
]

In [3]:
# Function to extract data from a single URL
def extract_data(url, expected_labels):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    labels = {label: "" for label in expected_labels}  # Initialize with empty strings
    
    for label_element, text_element in zip(soup.find_all(class_="label-infmlk"), soup.find_all(class_="text-infmlk")):
        label_text = label_element.find('div').get_text(strip=True)
        if label_text in expected_labels:
            labels[label_text] = text_element.find('div').get_text(strip=True)
    
    return pd.DataFrame([labels])

In [9]:
# Function to generate URL and extract data for a given code
def extract_data_of_given_code(given_code):
    if 10000 < given_code < 100000:
        url = f'https://www.melketabriz.ir/p/{given_code}'
        return extract_data(url, EXPECTED_LABELS)
    else:
        print('Invalid code')
        return None


In [5]:
# Function to add DataFrame to an existing CSV or create a new one
def add_to_csv(df, csv_file='combined_property_info.csv'):
    try:
        existing_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        existing_df = pd.DataFrame(columns=EXPECTED_LABELS)

    combined_df = pd.concat([existing_df, df], ignore_index=True).fillna(0)
    
    # Save the combined DataFrame to a CSV file with UTF-8 encoding
    combined_df.to_csv(csv_file, index=False, encoding='utf-8-sig')
    
    # Print the combined DataFrame to verify
    print(combined_df)

In [10]:
# Example usage
df = extract_data_of_given_code(23904)
if df is not None:
    add_to_csv(df)

  کد آگهی  منطقه محله متراژ             قیمت کل طبقه تعداد اتاق کل طبقات  \
0   23904  گلکار        159  ۶٬۷۷۶٬۰۰۰٬۰۰۰تومان               2        1   

  تعداد واحد در طبقه         موقعیت نوع ساخت سال ساخت  نوع سند زمان تحویل  \
0                     3 نبش یا 3 بر              1374  شش دانگ              

  متراژ گذر عرض قطعه متراژ زمین  
0         8     14.6        154  
